# Implementing backpropogation

### Loading and data cleanup

In [1]:
import numpy as np
import pandas as pd

In [2]:
admissions = pd.read_csv('binary.csv')
# create dummies for rank
data = pd.concat( [admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data = data.drop('rank', axis=1)
# standardize gre and gpa
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:, field] = (data[field]-mean)/std
# split off 10% data for testing
np.random.seed(42)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.ix[sample], data.drop(sample)
# split data into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, target_test = test_data.drop('admit', axis=1), test_data['admit']

### Implementing backpropogation

In [3]:
# activation function
def sigmoid(x):
    return 1/(1+np.exp(-x))

##### Hyperparameters

In [4]:
n_hidden = 3  # number of hidden units
epochs = 1500
learnrate = 0.05

In [5]:
n_records, n_features = features.shape
print("#records:", n_records)
print("#features:", n_features )

#records: 360
#features: 6


##### Initialize weights

In [6]:
weights_input_hidden = np.random.normal(scale=n_features ** -.5, size=(n_features, n_hidden))
print("Size input->hidden weights: %dx%d " % weights_input_hidden.shape)
weights_hidden_output = np.random.normal(scale=n_features ** -.5, size=n_hidden)
print("Size hidden->output weights: %d" % weights_hidden_output.shape)

Size input->hidden weights: 6x3 
Size hidden->output weights: 3


#### Run backpropogation to train

In [7]:
last_loss = None
for e in range(epochs):
    # step 1 - initialize weight steps for each layer
    del_w_input_hidden = np.zeros(weights_input_hidden.shape)
    del_w_hidden_output = np.zeros(weights_hidden_output.shape)
    
    for x,y in zip(features.values, targets):
        # forward pass
        hidden_input = np.dot(x, weights_input_hidden)
        hidden_activations = sigmoid(hidden_input)
        output = sigmoid( np.dot(hidden_activations, weights_hidden_output) )
        
        # backward pass
        # Calculate error gradient in output unit
        output_error = (y - output) * output * (1-output)

        # propagate errors to hidden layer
        hidden_error = np.dot(output_error, weights_hidden_output) * hidden_activations * (1 - hidden_activations)

        # Update the change in weights
        del_w_hidden_output += output_error * hidden_activations
        del_w_input_hidden += hidden_error * x[:, None]

    # Update weights
    weights_input_hidden += learnrate * del_w_input_hidden  / n_records
    weights_hidden_output += learnrate * del_w_hidden_output / n_records

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        hidden_activations = sigmoid(np.dot(x, weights_input_hidden))
        out = sigmoid(np.dot(hidden_activations, weights_hidden_output))
        loss = np.mean((out - targets) ** 2)
        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss

Train loss:  0.2537526908074026
Train loss:  0.23616508718457455
Train loss:  0.22757934488278436
Train loss:  0.22337303212847115
Train loss:  0.22128329110046635
Train loss:  0.22023939019060432
Train loss:  0.2197261014052837
Train loss:  0.21948781540899762
Train loss:  0.21939378424532516
Train loss:  0.21937537804742643


In [8]:
# Calculate accuracy on test data
hidden = sigmoid(np.dot(features_test, weights_input_hidden))
out = sigmoid(np.dot(hidden, weights_hidden_output))
predictions = out > 0.5
accuracy = np.mean(predictions == target_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

Prediction accuracy: 0.750
